In [60]:
from config import *

declared 4 variables


In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models
from torchvision import transforms

import pathlib
from PIL import Image

In [62]:
device = torch.device('cpu')
vgg = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
vgg = vgg.to(device)

In [63]:
for params in vgg.parameters():
    params.requires_grad = False

In [64]:
last_sequential_layer = list(vgg.children())[-1]
*list_of_layers, last_layer = list(last_sequential_layer)
in_features = last_layer.in_features

In [65]:
vgg.fc = nn.Linear(in_features, 6)
vgg.fc.requires_grad_ = True
vgg.classifier = nn.Sequential(*(list_of_layers + [vgg.fc]))

In [66]:
transform = transforms.Compose(
  [transforms.RandomHorizontalFlip(), #data augmentation
   transforms.ToTensor(),
   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])    
  ]
)

In [67]:
class SIGNSDataset():
    def __init__(self, base_dir: str | pathlib.Path, split: str = "train", transform = None) -> None:
        if isinstance(base_dir, str): base_dir = pathlib.Path(base_dir)
        path = base_dir.joinpath(split)
        files = path.iterdir()

        self.filenames = list(filter(lambda x: str(x).endswith(".jpg"), files))
        self.labels = [int(f.name[0]) for f in self.filenames]
        self.transform = transform
        
    def __len__(self):
        return len(self.filenames)
    def __getitem__(self, idx):
        image = Image.open(self.filenames[idx])
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]
class RunningMetric():
    def __init__(self) -> None:
        self.S = 0
        self.N = 0
    
    def update(self, val, size):
        self.S += val
        self.N += size

    def __call__(self):
        try:
            return self.S / float(self.N)
        except ZeroDivisionError as e:
            return 0
DIR = SIGNS_DIR.joinpath("64x64_SIGNS")
train_dataset = SIGNSDataset(DIR, "train_signs", transform=transform)
test_dataset = SIGNSDataset(DIR, "test_signs", transform=transform)
val_dataset = SIGNSDataset(DIR, "val_signs", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)
val_loader = DataLoader(val_dataset, batch_size=16)

In [68]:
loss_fn = nn.NLLLoss()
optimizer = optim.SGD(vgg.parameters(),lr=1e-3, momentum=0.9)

In [69]:
def train_eval(model: nn.Module, optimizer: optim.Optimizer, loss_fn , dataloaders: list, epochs: int = 10, lr: float = 0.01):
    for g in optimizer.param_groups:
        g['lr'] = lr
    
    train_loader, val_loader = dataloaders

    for epoch in range(1, epochs + 1):
        print("Epoch {} / {}".format(epoch, epochs))
        print("-"*10)

        running_loss = RunningMetric()
        running_ac  = RunningMetric()
        running_loss_v = RunningMetric()
        running_ac_v  = RunningMetric()

        for phase in ('train', 'val'):
            if phase == "train":
                model.train()
            else:
                model.eval()

            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    outputs1 = outputs.to('cpu')
                    _, preds = torch.max(outputs1, 1)
                    # preds = np.max(outputs1)
                    loss = loss_fn(outputs, targets)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                batch_size = inputs.size()[0]
                running_loss.update(batch_size*loss.item(),
                                    batch_size)
                running_ac.update(torch.sum(preds == targets.to('cpu')),
                                    batch_size)
                print("\r {} Loss: {:4f} - Accuracy {:4f}".format(phase, running_loss(), running_ac()), end="")
            print()

In [70]:
train_eval(vgg, optimizer=optimizer, loss_fn=loss_fn, dataloaders=[train_loader, val_loader], epochs=10, lr=1e-3)

Epoch 1 / 10
----------
 train Loss: -85.459944 - Accuracy 0.408565
 val Loss: -140.786461 - Accuracy 0.493056
Epoch 2 / 10
----------
 train Loss: -314.236378 - Accuracy 0.586250

KeyboardInterrupt: 